
# 전국 신규 민간 아파트 분양가격 동향

2013년부터 최근까지 부동산 가격 변동 추세가 아파트 분양가에도 반영될까요? 공공데이터 포털에 있는 데이터를 Pandas 의 melt, concat, pivot, transpose 와 같은 reshape 기능을 활용해 분석해 봅니다. 그리고 groupby, pivot_table, info, describe, value_counts 등을 통한 데이터 요약과 분석을 해봅니다. 이를 통해 전혀 다른 형태의 두 데이터를 가져와 정제하고 병합하는 과정을 다루는 방법을 알게 됩니다. 전처리 한 결과에 대해 수치형, 범주형 데이터의 차이를 이해하고 다양한 그래프로 시각화를 할 수 있게 됩니다.


## 다루는 내용

* 공공데이터를 활용해 전혀 다른 두 개의 데이터를 가져와서 전처리 하고 병합하기
* 수치형 데이터와 범주형 데이터를 바라보는 시각을 기르기
* 데이터의 형식에 따른 다양한 시각화 방법 이해하기

## 실습
* 공공데이터 다운로드 후 주피터 노트북으로 로드하기
* 판다스를 통해 데이터를 요약하고 분석하기
* 데이터 전처리와 병합하기
* 수치형 데이터와 범주형 데이터 다루기
* 막대그래프(bar plot), 선그래프(line plot), 산포도(scatter plot), 상관관계(lm plot), 히트맵, 상자수염그림, swarm plot, 도수분포표, 히스토그램(distplot) 실습하기

## 데이터 출처
* https://kosis.kr/statHtml/statHtml.do?orgId=414&tblId=DT_41401N_005&vw_cd=MT_ZTITLE&list_id=I1_3&seqNo=&lang_mode=ko&language=kor&obj_var_id=&itm_id=&conn_path=MT_ZTITLE
* 공공데이터 포털 : https://www.data.go.kr/data/15061057/fileData.do

### 전국 평균 분양가격(2013년 9월부터 2015년 8월까지)
* 전국 공동주택의 3.3제곱미터당 평균분양가격 데이터를 제공

###  주택도시보증공사_전국 평균 분양가격(2015년 10월~)
* 전국 공동주택의 연도별, 월별, 전용면적별 제곱미터당 평균분양가격 데이터를 제공
* 지역별 평균값은 단순 산술평균값이 아닌 가중평균값임

In [ ]:
# !pip install koreanize_matplotlib

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import koreanize_matplotlib

In [ ]:
df_last = pd.read_csv("data/apt-price-hug.csv", dtype={"시점":"object"}, encoding="cp949")
df_last.shape

In [ ]:
df_last.head()

In [ ]:
df_last.info()

### 결측치 보기
isnull 혹은 isna 를 통해 데이터가 비어있는지를 확인할 수 있습니다.
결측치는 True로 표시되는데, True == 1 이기 때문에 이 값을 다 더해주면 결측치의 수가 됩니다.

In [ ]:
df_last.isnull().sum()

In [ ]:
df_last.columns

In [ ]:
df_last.columns = ['시점', '전용면적', '지역구분', '지역명', '분양가']
df_last

In [ ]:
df_last = df_last[df_last["지역명"] != "소계"].copy()
df_last.shape

In [ ]:
df_last["지역명"].value_counts()

In [ ]:
df_last["지역구분"]

### 기초 기술 통계

In [ ]:
df_last.describe()

### 데이터 타입 변경
분양가격이 object(문자) 타입으로 되어 있습니다. 문자열 타입을 계산할 수 없기 때문에 수치 데이터로 변경해 줍니다. 결측치가 섞여 있을 때 변환이 제대로 되지 않습니다. 그래서 pd.to_numeric 을 통해 데이터의 타입을 변경합니다.

In [ ]:
df_last["분양가천원"] = pd.to_numeric(df_last["분양가"], errors="coerce")
df_last["분양가천원"]

### 평당분양가격 구하기
공공데이터포털에 올라와 있는 2013년부터의 데이터는 평당분양가격 기준으로 되어 있습니다.
분양가격을 평당기준으로 보기위해 3.3을 곱해서 "평당분양가격" 컬럼을 만들어 추가해 줍니다.

In [ ]:
df_last["평당분양가천원"] = df_last["분양가천원"] * 3.3

In [ ]:
df_last.describe()

In [ ]:
df_last.describe(include="object")

### 연도와 월을 분리하기
* pandas 의 string-handling 사용하기 : https://pandas.pydata.org/pandas-docs/stable/reference/series.html#string-handling

In [ ]:
df_last["연도"] = df_last["시점"].astype(str).str[:4].astype(int)
df_last["월"] = df_last["시점"].map(lambda x : str(x).split(".")[-1]).astype(int)
df_last

### 분양가격 요약하기

In [ ]:
df_last.describe()

### 필요없는 컬럼 제거하기
drop을 통해 전처리 해준 컬럼을 제거합니다. pandas의 데이터프레임과 관련된 메소드에는 axis 옵션이 필요할 때가 있는데 행과 열중 어떤 기준으로 처리를 할 것인지를 의미합니다. 보통 기본적으로 0으로 되어 있고 행을 기준으로 처리함을 의미합니다. 메모리 사용량이 줄어들었는지 확인합니다.

In [ ]:
# df_last = df_last.drop(columns=["지역구분", "시점"]).copy()
# df_last

## 최근 데이터 분석과 시각화 하기

In [ ]:
sns.barplot(data=df_last, x="연도", y="평당분양가천원")

## 2015년 8월 이전 데이터 보기

In [ ]:
df_first = pd.read_csv("data/apt-price-hug-201309-201508.csv", encoding="cp949")
df_first.shape

In [ ]:
df_first.head()

In [ ]:
df_first.tail()

In [ ]:
df_first.info()

In [ ]:
# 결측치가 있는지 봅니다.
df_first.isnull().sum().sum()

### melt로 Tidy data 만들기
pandas의 melt를 사용하면 데이터의 형태를 변경할 수 있습니다. 
df_first 변수에 담긴 데이터프레임은 df_last에 담겨있는 데이터프레임의 모습과 다릅니다. 
같은 형태로 만들어주어야 데이터를 합칠 수 있습니다. 
데이터를 병합하기 위해 melt를 사용해 열에 있는 데이터를 행으로 녹여봅니다.

<img src="https://pandas.pydata.org/docs/_images/reshaping_melt.png" width="500">

* https://pandas.pydata.org/docs/user_guide/reshaping.html#reshaping-by-melt
* [Tidy Data 란?](https://vita.had.co.nz/papers/tidy-data.pdf)

In [ ]:
# pd.melt 를 사용하며, 녹인 데이터는 df_first_melt 변수에 담습니다. 
df_first_melt = pd.melt(df_first, id_vars="지역")
df_first_melt.head()

In [ ]:
# df_first_melt 변수에 담겨진 컬럼의 이름을 
# ["지역명", "기간", "평당분양가격"] 으로 변경합니다.
df_first_melt.columns = ["지역명", "기간", "평당분양가천원"]
df_first_melt.head(1)

### 연도와 월을 분리하기
* pandas 의 string-handling 사용하기 : https://pandas.pydata.org/pandas-docs/stable/reference/series.html#string-handling

In [ ]:
# str.split 을 통해 문자열 분리가 가능합니다.
# "2013년12월" 이라는 문자열에서 연도와 월을 추출하기 "년"이라는 글자로 나누고 앞에 있는 글자를 가져오면 연도가 됩니다.
# 그리고 월은 int 타입으로 변경합니다.

df_first_melt["연도"] = df_first_melt["기간"].str.split("년", expand=True)[0]
df_first_melt["연도"] = df_first_melt["연도"].astype(int)
df_first_melt.head(1)

In [ ]:
df_first_melt["월"] = df_first_melt["기간"].str.split("년", expand=True)[1]
df_first_melt["월"] = df_first_melt["월"].str.replace("월", "")
df_first_melt["월"] = df_first_melt["월"].astype(int)
df_first_melt["월"]

In [ ]:
cols = ['지역명', '연도', '월', '평당분양가천원']
cols

In [ ]:
df_last_prepare = df_last[df_last["전용면적"] == "전체"].copy()
df_last_prepare = df_last_prepare[cols].copy()
df_last_prepare.head()

In [ ]:
df_last

In [ ]:
df_last_prepare = df_last[df_last["전용면적"] == "전체"].copy()
df_last_prepare = df_last_prepare[cols].copy()
df_last_prepare.head()

In [ ]:
# df_first_melt에서 공통된 컬럼만 가져온 뒤
# copy로 복사해서 df_first_prepare 변수에 담습니다.
df_first_prepare = df_first_melt[cols].copy()
df_first_prepare.head()

In [ ]:
df_first_prepare = df_first_melt[cols].copy()
df_first_prepare.head()

### concat 으로 데이터 합치기
<img src="https://pandas.pydata.org/pandas-docs/stable/_images/merging_concat_basic.png">

* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html

In [ ]:
# df_first_prepare 와 df_last_prepare 를 합쳐줍니다.
df = pd.concat([df_first_prepare, df_last_prepare], ignore_index=True)
df.shape

In [ ]:
# 제대로 합쳐졌는지 미리보기를 합니다.
df

In [ ]:
# 연도별로 데이터가 몇개씩 있는지 value_counts를 통해 세어봅니다.



## 2013년부터 최근 데이터까지 시각화하기
### 연도별 평당분양가격 보기

In [ ]:
# barplot 으로 연도별 평당분양가천원 그리기
# sns.barplot(data=df, x="연도", y="평당분양가천원")

In [ ]:
# df.groupby("연도")["평당분양가천원"].describe()

In [ ]:
# sns.boxplot(data=df, x="연도", y="평당분양가천원")

### 지역별 평당 분양가격 보기

In [ ]:
# sns.barplot(data=df, x="지역명", y="평당분양가천원")